In [45]:
#import shapefile
import pandas as pd
import random
import numpy as np
#import Data_encoding





def COUNT_LINK_by_NODE(L_data, N_data):
    new_data = pd.DataFrame(columns=['Source', 'num_of_Targets', 'Targets'])
    for name, group in L_data.groupby('Source'):
        print(group)
    #new_data.to_csv('노드별_도착지점_정보.csv', encoding='cp949')

try:
    node_data = pd.read_csv('중구_nodes_200331_1.csv', encoding='cp949')
    link_data = pd.read_csv('중구_links_200331_1.csv', encoding='cp949')
except:
    node, link = Data_encoding.Encoding().data_parsing()
    node.rename(columns={'REMARK': 'NUM_of_PATH'}, inplace=True)
    node.to_csv('중구_nodes_200331_1.csv', encoding='cp949')
    link.to_csv('중구_links_200331_1.csv', encoding='cp949')
    node_data = pd.read_csv('중구_nodes_200331_1.csv', encoding='cp949')
    link_data = pd.read_csv('중구_links_200331_1.csv', encoding='cp949')


print(node_data[:30][['Id', 'NODE_NAME']])
print(link_data[:6][['Source', 'Target', 'ROAD_NAME']])
#COUNT_LINK_by_NODE(link_data, node_data)




            Id     NODE_NAME
0   1010000300         후암삼거리
1   1010000301   SK커뮤니케이션즈본사
2   1010000302       LG이노텍본사
3   1010000400       약수역앞교차로
4   1010000600      남산2호터널입구
5   1010000700         도동삼거리
6   1010000900      남산3호터널북측
7   1010001000           서울역
8   1010001100      남산1호터널북측
9   1010002800       중구청앞교차로
10  1010003200     중부경찰서앞교차로
11  1010003201     을지로3가역교차로
12  1010003700    정동제일교회앞교차로
13  1010004000        시청앞교차로
14  1010004800       한국체육박물관
15  1010004900         프레스센터
16  1010005300         신평화시장
17  1010005400  남산공원장충유아숲체험장
18  1010005500      국립극장앞교차로
19  1010005600        금호터널북측
20  1010005700        약수고가차도
21  1010005800        약수고가차도
22  1010005900        남산학사빌딩
23  1010006000      약수고가차도남측
24  1010006001    약수역(서울3호선)
25  1010006002     신당동373-26
26  1010006400      남산공원길삼거리
27  1010006500         남산삼거리
28  1010006700     서울역고가차도서측
29  1010006900      남산2호터널입구
   Source  Target ROAD_NAME
0       0       1     소월로2길
1       1       0     소월로2길
2       1    -100

In [46]:
def rargmax(vector):
    m = np.amax(vector)
    indices = np.nonzero(vector==m)[0]
    return random.choice(indices)

In [86]:
class env():
    def __init__(self):
        self.state_list=[]
        self.action_list=[]
        self.reward_list1=[]
        self.reward_list2=[]
        self.reward_list3=[]
        self.reward_list4=[]
        self.env_create()
        self.create_reward()
        self.curr_state=10
        self.epilson=0.99
        self.decay_epilson=0.9995
        self.Q = np.zeros([4,310,6])
        self.lr = 0.001
        self.y = 0.9
        self.path=[]
        self.goal_num=0
        
    def env_create(self):
        node_data = pd.read_csv('중구_nodes_200331_1.csv', encoding='cp949')
        link_data = pd.read_csv('중구_links_200331_1.csv', encoding='cp949')
        node_data=node_data.drop(node_data.columns[[0]], axis='columns')
        node_data=node_data.drop(node_data.columns[[1,2,3,4]], axis='columns')
        link_data=link_data.drop(link_data.columns[[0,1]], axis='columns')
        link_data=link_data.drop(link_data.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15]], axis='columns')
        node=node_data.values
        link=link_data.values
        link_num=0
        action_list= [[0 for j in range(6)] for i in range(310)]
        for i in range(310):
            for j in range(6):
                if link[link_num][0]==i:
                    action_list[i][j]=link[link_num][1]
                    link_num=link_num+1
                    if link_num>=len(link):
                        break
                else:
                    action_list[i][j]=-100
        for k in range(1,6,1):
            action_list[309][k]=-100
        state_list= [[0 for j in range(3)] for i in range(310)]
        for i in range(310):
            state_list[i][1]=node[i][1]
            state_list[i][2]=node[i][2]
        self.state_list=state_list
        self.action_list=action_list
        
    def create_reward(self):
        self.reward_list1=[0 for i in range(310)]
        self.reward_list2=[0 for i in range(310)]
        self.reward_list3=[0 for i in range(310)]
        self.reward_list4=[0 for i in range(310)]
        #self.reward_list1[226]=100
        self.reward_list1[308]=100
        self.reward_list2[306]=100
        self.reward_list3[141]=100
        self.reward_list4[10]=100
    
    def act(self,action):
        done=False
        next_state=self.action_list[self.curr_state][action]
        if next_state==-100:
            next_state=self.curr_state
        self.curr_state=next_state
        if self.goal_num==0:
            reward=self.reward_list1[next_state]
        elif self.goal_num==1:
            reward=self.reward_list2[next_state]
        elif self.goal_num==2:
            reward=self.reward_list3[next_state]
        elif self.goal_num==3:
            reward=self.reward_list4[next_state]
        if reward>=1:
            if self.goal_num!=3:
                self.goal_num=self.goal_num+1
            else:
                done=True
        return next_state,reward,done
    
    def reset(self):
        self.curr_state=10
        self.goal_num=0
        
    def qlearning(self):
        for i in range(5000):
            self.reset()
            s=10
            rAll = 0
            reward=0
            tp=0
            print("num_episdoe",i,self.epilson)
            times=15000
            done=False
            for time in range(times):
                if time==14999:
                    print("fail")
                if self.epilson >= random.random():
                    a=np.random.randint(0,6)
                else:
                    a = rargmax(self.Q[tp,s,:]) #+ np.random.randn(1,4)*(20))
                s1,reward,done=self.act(a)
                #print(s,s1,reward)
                #s1=s1[1]
                self.Q[tp,s,a] = self.Q[tp,s,a] + self.lr*(reward + self.y*np.max(self.Q[tp,s1,:]) - self.Q[tp,s,a])
                #s,a]= timeW*Q1[s,a] + treasureW*Q2[s,a]
                if reward>=1:
                    print(s1,reward,time)
                    if tp!=3:
                        tp=tp+1
                    else:
                        break
                s = s1
            if self.epilson>0.01:
                self.epilson=self.epilson*self.decay_epilson
        print("train stop")
        self.reset()
        s=10
        tp1=0
        final_step=0
        while(True):
            final_step=final_step+1
            self.path.append([self.state_list[s][1],self.state_list[s][2]])
            a = rargmax(self.Q[tp1,s,:])
            s1,reward,done=self.act(a)
            if reward>=1:
                print(s1,reward,done)
                if tp1!=3:
                    tp1=tp1+1
                else:
                    break
            s=s1

        
        

In [87]:
patrol_env=env()
print(patrol_env.state_list)
print(patrol_env.action_list)

[[0, 37.5529871, 126.97653400000002], [0, 37.55428955, 126.9765508], [0, 37.55352886, 126.97527279999998], [0, 37.55482232, 127.01064939999999], [0, 37.55531959, 127.00386110000001], [0, 37.55600424, 126.9769403], [0, 37.55540491, 126.98315500000001], [0, 37.55705081, 126.97262420000001], [0, 37.55570512, 126.99298390000001], [0, 37.56435007, 126.99834579999998], [0, 37.5648325, 126.98996480000001], [0, 37.56584413, 126.9897968], [0, 37.56526116, 126.97341000000002], [0, 37.56598487, 126.9790486], [0, 37.566976600000004, 126.97919809999999], [0, 37.56705463, 126.9771277], [0, 37.56964232, 127.0122029], [0, 37.55082092, 126.9988289], [0, 37.55128848, 127.00064509999999], [0, 37.55185485, 127.0130668], [0, 37.55215176, 127.0121868], [0, 37.55218456, 127.01270359999998], [0, 37.55261793, 126.9968071], [0, 37.55276081, 127.01229440000002], [0, 37.55385312, 127.01142279999999], [0, 37.55317832, 127.0119613], [0, 37.55455528, 126.9831145], [0, 37.55490657, 126.9812592], [0, 37.55484714, 126.

In [88]:
patrol_env.qlearning()

num_episdoe 0 0.99
fail
num_episdoe 1 0.9895050000000001
308 100 803
fail
num_episdoe 2 0.9890102475000001
fail
num_episdoe 3 0.9885157423762502
fail
num_episdoe 4 0.9880214845050621
fail
num_episdoe 5 0.9875274737628096
308 100 1134
fail
num_episdoe 6 0.9870337100259282
308 100 826
fail
num_episdoe 7 0.9865401931709153
fail
num_episdoe 8 0.98604692307433
fail
num_episdoe 9 0.9855538996127928
fail
num_episdoe 10 0.9850611226629865
308 100 886
306 100 1113
141 100 3897
fail
num_episdoe 11 0.984568592101655
308 100 1328
fail
num_episdoe 12 0.9840763078056043
fail
num_episdoe 13 0.9835842696517015
fail
num_episdoe 14 0.9830924775168758
fail
num_episdoe 15 0.9826009312781174
fail
num_episdoe 16 0.9821096308124784
308 100 347
306 100 1418
141 100 1571
fail
num_episdoe 17 0.9816185759970721
308 100 390
306 100 816
141 100 1547
10 100 1904
num_episdoe 18 0.9811277667090736
fail
num_episdoe 19 0.9806372028257192
308 100 1145
fail
num_episdoe 20 0.9801468842243064
fail
num_episdoe 21 0.97965681

10 100 924
num_episdoe 138 0.9239774692136565
308 100 190
306 100 299
141 100 361
10 100 787
num_episdoe 139 0.9235154804790497
308 100 1379
306 100 1672
141 100 1788
10 100 1920
num_episdoe 140 0.9230537227388103
fail
num_episdoe 141 0.9225921958774409
308 100 310
306 100 393
141 100 912
10 100 1014
num_episdoe 142 0.9221308997795022
308 100 656
306 100 1696
fail
num_episdoe 143 0.9216698343296126
308 100 272
306 100 1560
141 100 1665
10 100 1981
num_episdoe 144 0.9212089994124478
308 100 90
306 100 370
141 100 497
10 100 537
num_episdoe 145 0.9207483949127416
308 100 337
fail
num_episdoe 146 0.9202880207152853
308 100 346
306 100 1188
141 100 1370
10 100 1429
num_episdoe 147 0.9198278767049277
308 100 68
306 100 585
141 100 743
10 100 1027
num_episdoe 148 0.9193679627665753
fail
num_episdoe 149 0.9189082787851921
308 100 1266
306 100 2256
141 100 2310
10 100 2464
num_episdoe 150 0.9184488246457996
308 100 1275
306 100 1726
141 100 2248
10 100 2912
num_episdoe 151 0.9179896002334768
3

10 100 475
num_episdoe 244 0.8762701492581205
308 100 342
306 100 666
141 100 905
10 100 1303
num_episdoe 245 0.8758320141834915
308 100 145
306 100 351
141 100 476
10 100 507
num_episdoe 246 0.8753940981763999
308 100 128
306 100 228
141 100 375
10 100 683
num_episdoe 247 0.8749564011273118
308 100 609
306 100 762
141 100 939
10 100 1021
num_episdoe 248 0.8745189229267482
308 100 138
306 100 1278
141 100 1377
10 100 1590
num_episdoe 249 0.8740816634652848
308 100 361
306 100 951
141 100 999
10 100 1068
num_episdoe 250 0.8736446226335522
308 100 113
306 100 209
141 100 388
10 100 495
num_episdoe 251 0.8732078003222356
308 100 365
306 100 486
141 100 651
10 100 793
num_episdoe 252 0.8727711964220745
308 100 198
306 100 334
141 100 539
10 100 580
num_episdoe 253 0.8723348108238634
308 100 97
306 100 187
141 100 300
10 100 648
num_episdoe 254 0.8718986434184516
308 100 403
306 100 494
141 100 554
10 100 623
num_episdoe 255 0.8714626940967424
308 100 316
306 100 364
141 100 609
10 100 812


10 100 340
num_episdoe 350 0.8310260802509836
308 100 153
306 100 256
141 100 392
10 100 494
num_episdoe 351 0.8306105672108581
308 100 155
306 100 260
141 100 333
10 100 396
num_episdoe 352 0.8301952619272528
308 100 390
306 100 422
141 100 565
10 100 636
num_episdoe 353 0.8297801642962892
308 100 222
306 100 340
141 100 562
10 100 596
num_episdoe 354 0.8293652742141411
308 100 191
306 100 216
141 100 356
10 100 426
num_episdoe 355 0.8289505915770341
308 100 171
306 100 262
141 100 458
10 100 491
num_episdoe 356 0.8285361162812457
308 100 52
306 100 102
141 100 148
10 100 259
num_episdoe 357 0.8281218482231051
308 100 182
306 100 283
141 100 642
10 100 673
num_episdoe 358 0.8277077872989935
308 100 101
306 100 154
141 100 219
10 100 266
num_episdoe 359 0.8272939334053441
308 100 105
306 100 271
141 100 354
10 100 414
num_episdoe 360 0.8268802864386414
308 100 57
306 100 170
141 100 272
10 100 416
num_episdoe 361 0.8264668462954221
308 100 130
306 100 185
141 100 335
10 100 352
num_epi

308 100 140
306 100 198
141 100 228
10 100 296
num_episdoe 452 0.789696286669329
308 100 78
306 100 189
141 100 344
10 100 425
num_episdoe 453 0.7893014385259944
308 100 114
306 100 217
141 100 310
10 100 393
num_episdoe 454 0.7889067878067315
308 100 59
306 100 172
141 100 248
10 100 374
num_episdoe 455 0.7885123344128282
308 100 125
306 100 237
141 100 266
10 100 313
num_episdoe 456 0.7881180782456219
308 100 178
306 100 200
141 100 241
10 100 303
num_episdoe 457 0.787724019206499
308 100 190
306 100 234
141 100 291
10 100 310
num_episdoe 458 0.7873301571968958
308 100 303
306 100 341
141 100 365
10 100 401
num_episdoe 459 0.7869364921182974
308 100 63
306 100 273
141 100 335
10 100 358
num_episdoe 460 0.7865430238722383
308 100 145
306 100 232
141 100 316
10 100 414
num_episdoe 461 0.7861497523603023
308 100 152
306 100 190
141 100 239
10 100 270
num_episdoe 462 0.7857566774841221
308 100 74
306 100 102
141 100 195
10 100 251
num_episdoe 463 0.7853637991453801
308 100 82
306 100 120

...

308 100 17
306 100 26
141 100 36
10 100 45
num_episdoe 4443 0.10730222470591443
308 100 16
306 100 25
141 100 36
10 100 45
num_episdoe 4444 0.10724857359356148
308 100 15
306 100 24
141 100 42
10 100 50
num_episdoe 4445 0.10719494930676471
308 100 19
306 100 29
141 100 41
10 100 51
num_episdoe 4446 0.10714135183211133
308 100 16
306 100 26
141 100 36
10 100 45
num_episdoe 4447 0.10708778115619529
308 100 20
306 100 31
141 100 41
10 100 50
num_episdoe 4448 0.1070342372656172
308 100 17
306 100 28
141 100 38
10 100 49
num_episdoe 4449 0.1069807201469844
308 100 14
306 100 23
141 100 33
10 100 47
num_episdoe 4450 0.10692722978691091
308 100 18
306 100 27
141 100 37
10 100 46
num_episdoe 4451 0.10687376617201746
308 100 14
306 100 24
141 100 34
10 100 42
num_episdoe 4452 0.10682032928893147
308 100 18
306 100 27
141 100 37
10 100 45
num_episdoe 4453 0.10676691912428701
308 100 16
306 100 25
141 100 45
10 100 53
num_episdoe 4454 0.10671353566472487
308 100 19
306 100 29
141 100 39
10 100 47

10 100 48
num_episdoe 4547 0.10186377471052772
308 100 16
306 100 25
141 100 35
10 100 45
num_episdoe 4548 0.10181284282317246
308 100 16
306 100 25
141 100 35
10 100 43
num_episdoe 4549 0.10176193640176089
308 100 16
306 100 26
141 100 36
10 100 44
num_episdoe 4550 0.10171105543356002
308 100 19
306 100 30
141 100 41
10 100 50
num_episdoe 4551 0.10166019990584324
308 100 14
306 100 23
141 100 36
10 100 48
num_episdoe 4552 0.10160936980589032
308 100 14
306 100 23
141 100 35
10 100 43
num_episdoe 4553 0.10155856512098738
308 100 19
306 100 28
141 100 40
10 100 48
num_episdoe 4554 0.10150778583842689
308 100 15
306 100 25
141 100 37
10 100 50
num_episdoe 4555 0.10145703194550768
308 100 16
306 100 25
141 100 36
10 100 47
num_episdoe 4556 0.10140630342953492
308 100 14
306 100 26
141 100 43
10 100 52
num_episdoe 4557 0.10135560027782016
308 100 17
306 100 26
141 100 40
10 100 48
num_episdoe 4558 0.10130492247768126
308 100 16
306 100 25
141 100 35
10 100 43
num_episdoe 4559 0.10125427001

num_episdoe 4658 0.09636301816061112
308 100 14
306 100 23
141 100 43
10 100 51
num_episdoe 4659 0.09631483665153082
308 100 17
306 100 29
141 100 40
10 100 49
num_episdoe 4660 0.09626667923320506
308 100 17
306 100 26
141 100 37
10 100 45
num_episdoe 4661 0.09621854589358847
308 100 14
306 100 23
141 100 33
10 100 41
num_episdoe 4662 0.09617043662064167
308 100 15
306 100 24
141 100 34
10 100 44
num_episdoe 4663 0.09612235140233136
308 100 14
306 100 25
141 100 35
10 100 43
num_episdoe 4664 0.0960742902266302
308 100 17
306 100 28
141 100 40
10 100 50
num_episdoe 4665 0.09602625308151688
308 100 16
306 100 26
141 100 37
10 100 48
num_episdoe 4666 0.09597823995497613
308 100 15
306 100 25
141 100 36
10 100 44
num_episdoe 4667 0.09593025083499865
308 100 15
306 100 25
141 100 35
10 100 51
num_episdoe 4668 0.09588228570958116
308 100 18
306 100 27
141 100 37
10 100 45
num_episdoe 4669 0.09583434456672638
308 100 15
306 100 26
141 100 36
10 100 44
num_episdoe 4670 0.09578642739444301
308 

308 100 14
306 100 23
141 100 33
10 100 41
num_episdoe 4772 0.09102263794641416
308 100 16
306 100 25
141 100 37
10 100 46
num_episdoe 4773 0.09097712662744095
308 100 14
306 100 23
141 100 34
10 100 42
num_episdoe 4774 0.09093163806412724
308 100 15
306 100 24
141 100 36
10 100 44
num_episdoe 4775 0.09088617224509518
308 100 14
306 100 23
141 100 33
10 100 41
num_episdoe 4776 0.09084072915897264
308 100 14
306 100 23
141 100 39
10 100 47
num_episdoe 4777 0.09079530879439315
308 100 17
306 100 26
141 100 38
10 100 48
num_episdoe 4778 0.09074991113999596
308 100 15
306 100 28
141 100 38
10 100 46
num_episdoe 4779 0.09070453618442596
308 100 14
306 100 25
141 100 35
10 100 43
num_episdoe 4780 0.09065918391633375
308 100 17
306 100 27
141 100 37
10 100 46
num_episdoe 4781 0.09061385432437559
308 100 17
306 100 26
141 100 36
10 100 44
num_episdoe 4782 0.0905685473972134
308 100 19
306 100 28
141 100 38
10 100 46
num_episdoe 4783 0.0905232631235148
308 100 15
306 100 26
141 100 36
10 100 44

308 100 14
306 100 23
141 100 33
10 100 42
num_episdoe 4905 0.08516509047633747
308 100 16
306 100 27
141 100 37
10 100 45
num_episdoe 4906 0.08512250793109931
308 100 17
306 100 26
141 100 36
10 100 44
num_episdoe 4907 0.08507994667713377
308 100 18
306 100 27
141 100 37
10 100 45
num_episdoe 4908 0.08503740670379521
308 100 14
306 100 23
141 100 39
10 100 47
num_episdoe 4909 0.08499488800044332
308 100 15
306 100 26
141 100 36
10 100 44
num_episdoe 4910 0.0849523905564431
308 100 16
306 100 25
141 100 44
10 100 53
num_episdoe 4911 0.08490991436116488
308 100 15
306 100 24
141 100 38
10 100 47
num_episdoe 4912 0.0848674594039843
308 100 16
306 100 26
141 100 37
10 100 46
num_episdoe 4913 0.08482502567428231
308 100 16
306 100 26
141 100 36
10 100 44
num_episdoe 4914 0.08478261316144517
308 100 14
306 100 23
141 100 33
10 100 43
num_episdoe 4915 0.08474022185486445
308 100 14
306 100 23
141 100 33
10 100 41
num_episdoe 4916 0.08469785174393701
308 100 16
306 100 25
141 100 35
10 100 43

In [91]:
patrol_env.path.append([37.5648325, 126.98996480000001])
print(patrol_env.path)
print(len(patrol_env.path))
print(patrol_env.state_list[308][1],patrol_env.state_list[308][2])
print(patrol_env.state_list[306][1],patrol_env.state_list[306][2])
print(patrol_env.state_list[141][1],patrol_env.state_list[141][2])

[[37.5648325, 126.98996480000001], [37.56436928, 126.99285970000001], [37.56425452, 126.995792], [37.56435007, 126.99834579999998], [37.56463798, 127.0024376], [37.56686931, 127.002264], [37.5666292, 127.0044568], [37.56804795, 127.0072386], [37.56883976, 127.00964979999999], [37.56824554, 127.0109598], [37.56789346, 127.01252609999999], [37.56801656, 127.0160061], [37.56837118, 127.01963959999999], [37.56568495, 127.01969540000002], [37.56564997, 127.0211361], [37.56618848, 127.02110829999998], [37.56564997, 127.0211361], [37.56568495, 127.01969540000002], [37.56537042, 127.01619869999999], [37.56313919, 127.0154705], [37.56027516, 127.01378100000001], [37.55482232, 127.01064939999999], [37.55385312, 127.01142279999999], [37.55317832, 127.0119613], [37.55335658, 127.0110843], [37.55317832, 127.0119613], [37.55276081, 127.01229440000002], [37.55645612, 127.0093844], [37.55892239, 127.0067558], [37.5590977, 127.00584520000001], [37.55910974, 127.0051006], [37.55972716, 127.0028567000000

In [90]:
import folium
center = [37.5648325, 126.9899648]
m = folium.Map(location=center, zoom_start=15)
folium.PolyLine(locations=patrol_env.path,
               tooltip = 'PolyLine').add_to(m)
m